### Extraction of title from pdf

In [27]:
import PyPDF2

In [28]:
pdf_path = '2306.06804.pdf'

In [29]:
# Extract title of a pdf

def extract_title(pdf_file):

  pdf_reader = PyPDF2.PdfReader(pdf_file)
  document_info = pdf_reader.metadata
  #print(document_info)
  pdf_title = document_info.get('/Title')
  return pdf_title

In [30]:
title = extract_title(pdf_path)

In [31]:
title, type(title)

('', PyPDF2.generic._base.TextStringObject)

In [32]:
# Looking for fontsize and fontname of each of lines within pdf
import fitz

def scrape(filePath):
    results = [] # list of tuples that store the information as (text, font size, font name) 
    pdf = fitz.open(filePath) # filePath is a string that contains the path to the pdf
    for page in pdf:
        dict = page.get_text("dict")
        blocks = dict["blocks"]
        for block in blocks:
            if "lines" in block.keys():
                spans = block['lines']
                for span in spans:
                    data = span['spans']
                    for lines in data:
                            results.append((lines['text'], lines['size'], lines['font']))
                            # lines['text'] -> string, lines['size'] -> font size, lines['font'] -> font name
    pdf.close()
    return results

In [33]:
output = scrape(pdf_path)
output

[('Neural Machine Translation for the Indigenous Languages of the',
  14.346199989318848,
  'NimbusRomNo9L-Medi'),
 ('Americas: An Introduction.', 14.346199989318848, 'NimbusRomNo9L-Medi'),
 ('Manuel Mager', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('♡', 7.970099925994873, 'CMSY8'),
 ('∗', 7.970099925994873, 'CMSY8'),
 (' ', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Rajat Bhatnagar', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('♠', 7.970099925994873, 'CMSY8'),
 ('Graham Neubig', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('♯', 7.970099925994873, 'CMMI8'),
 ('Ngoc Thang Vu', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('♢', 7.970099925994873, 'CMSY8'),
 (' ', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Katharina Kann', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('♠', 7.970099925994873, 'CMSY8'),
 ('♡', 7.970099925994873, 'CMSY8'),
 ('AWS AI Labs', 11.9552001953125, 'NimbusRomNo9L-Regu'),
 ('♯', 7.970099925994873, 'CMMI8'),
 ('Carnegie Mellon University', 11.9552001953125, 'NimbusRomNo9L-R

In [35]:
# Checking is Title was extracted correctly
if title is None or title == '':
      new_title = ''
      max_font_size = max(output, key=lambda x: x[1])[1]
      elements_with_max_font = [element for element in output if element[1] == max_font_size]

      print("PDF title is:\n")
      for element in elements_with_max_font:
            new_title += ' ' + element[0]
      print(new_title)
else:
      new_title = title
      print(new_title)

PDF title is:

 DATA


In [24]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize, word_tokenize

stop_words = set(stopwords.words('english'))


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     WRONG_VERSION_NUMBER] wrong version number
[nltk_data]     (_ssl.c:1122)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     WRONG_VERSION_NUMBER] wrong version number
[nltk_data]     (_ssl.c:1122)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     WRONG_VERSION_NUMBER] wrong version number
[nltk_data]     (_ssl.c:1122)>


In [25]:
# pre-process the extracted text

def preprocess_text(text):

  sentences = sent_tokenize(text)  # Split into sentences
  tokens = [word_tokenize(sentence.lower()) for sentence in sentences]  # Tokenize each sentence
  preprocessed_tokens = []
  for sentence_tokens in tokens:
    filtered_tokens = [token for token in sentence_tokens if token not in stop_words]  # Remove stop words
    preprocessed_tokens.extend(filtered_tokens)
  return preprocessed_tokens

In [26]:
clean_title = preprocess_text(new_title)
print(clean_title)

[]


In [48]:
import string

def remove_punctuations(text):

  punctuations = string.punctuation + "/_*"

  no_punct_text = " ".join([c for c in text if c not in punctuations])

  return no_punct_text

clean_title = remove_punctuations(clean_title)
print(f"Text without punctuations: {clean_title}")

Text without punctuations: 


### Extraction of text from other sections of pdf

##### Extracting section headers using fontsize and fontname attributes

In [49]:
# Finding the fontsize and fontname of 'Abstract'
abstract_index = next(i for i, item in enumerate(output) if item[0] == "Abstract " or item[0] == 'ABSTRACT ' or item[0] == "Abstract" or item[0] == 'ABSTRACT')

font_size = output[abstract_index][1]
font_name = output[abstract_index][2]
print(font_size, font_name)

11.9552001953125 NimbusRomNo9L-Medi


In [50]:
filtered_items = [item for item in output if item[1] == font_size and item[2] == font_name]
filtered_items = filtered_items[:-1] # Excluding references from headers
filtered_items

[('Manuel Mager', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 (' ', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Rajat Bhatnagar', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Graham Neubig', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Ngoc Thang Vu', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 (' ', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Katharina Kann', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Abstract', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('1', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Introduction', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('2', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Background and Definitions', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('3', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Challenges and open questions', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('4', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Available MT datasets for ILA', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('5', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Low-resource MT', 11

In [51]:
# Removing unwanted parts of section headers
new_abstract_index = next((i for i, item in enumerate(filtered_items) if item[0] == 'Abstract 'or item[0] == 'ABSTRACT' 
                           or item[0] == 'Abstract' or item[0] == 'ABSTRACT '), None)

new_items = filtered_items[new_abstract_index:] if new_abstract_index is not None else filtered_items

new_con_index = next((i for i, element in enumerate(new_items) if element[0] == 'CONCLUSIONS' or element[0] == 'CONCLUSIONS ' 
                      or element[0] == 'Conclusions' or element[0] == 'Conclusions ' or element[0] == 'Conclusion'
                      or element[0] == 'Conclusion '), None)

new_items = new_items[:new_con_index + 1]

new_items

[('Abstract', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('1', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Introduction', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('2', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Background and Definitions', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('3', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Challenges and open questions', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('4', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Available MT datasets for ILA', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('5', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Low-resource MT', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('6', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Low-resource MT paradigms', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('7', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('Advances in MT for the indigenous', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('languages of the Americas', 11.9552001953125, 'NimbusRomNo9L-Medi'),
 ('8', 11.9552001953125, 'NimbusRomNo9L

In [52]:
# Parse the entire text from pdf

def extract_text_from_pdf(pdf_file_path):
    """Extracts text from a PDF file and returns it as a string."""

    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)

        full_text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            full_text += page_text

    return full_text

In [53]:
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)

Neural Machine Translation for the Indigenous Languages of the
Americas: An Introduction.
Manuel Mager♡∗Rajat Bhatnagar♠Graham Neubig♯
Ngoc Thang Vu♢Katharina Kann♠
♡AWS AI Labs♯Carnegie Mellon University
♠University of Colorado Boulder♢University of Stuttgart
Abstract
Neural models have drastically advanced state
of the art for machine translation (MT) be-
tween high-resource languages. Traditionally,
these models rely on large amounts of train-
ing data, but many language pairs lack these
resources. However, an important part of the
languages in the world do not have this amount
of data. Most languages from the Americas are
among them, having a limited amount of par-
allel and monolingual data, if any. Here, we
present an introduction to the interested reader
to the basic challenges, concepts, and tech-
niques that involve the creation of MT systems
for these languages. Finally, we discuss the
recent advances and findings and open ques-
tions, product of an increased interest of the


In [54]:
# Removing non-ascii characters, urls from extracted text

import re
text_without_non_ascii = re.sub(r"[^\x00-\x7F]", "", extracted_text) 
text_without_non_ascii = re.sub(r",.-/:","",text_without_non_ascii)
text_without_urls = re.sub(r"h/t_tps?://[^\s]+","",text_without_non_ascii)

print(text_without_urls)

Neural Machine Translation for the Indigenous Languages of the
Americas: An Introduction.
Manuel MagerRajat BhatnagarGraham Neubig
Ngoc Thang VuKatharina Kann
AWS AI LabsCarnegie Mellon University
University of Colorado BoulderUniversity of Stuttgart
Abstract
Neural models have drastically advanced state
of the art for machine translation (MT) be-
tween high-resource languages. Traditionally,
these models rely on large amounts of train-
ing data, but many language pairs lack these
resources. However, an important part of the
languages in the world do not have this amount
of data. Most languages from the Americas are
among them, having a limited amount of par-
allel and monolingual data, if any. Here, we
present an introduction to the interested reader
to the basic challenges, concepts, and tech-
niques that involve the creation of MT systems
for these languages. Finally, we discuss the
recent advances and findings and open ques-
tions, product of an increased interest of the
NLP commun

In [55]:
# Format the section headings for extraction of text under each section
output = []
current_element = ''

for i, item in enumerate(new_items):
    if i > 0 and item[0].isdigit() and not new_items[i - 1][0].isdigit():

        output.append(current_element.strip())
        current_element = item[0] 
    else:
        current_element += ' ' + item[0]

# Add the last element to the output
if current_element:
    output.append(current_element.strip())

print(output)

['Abstract', '1 Introduction', '2 Background and Definitions', '3 Challenges and open questions', '4 Available MT datasets for ILA', '5 Low-resource MT', '6 Low-resource MT paradigms', '7 Advances in MT for the indigenous languages of the Americas', '8 Ethical aspects', '9 Conclusion']


In [56]:
new_output = []

if(len(output[0].split(' '))>1):
    words = output[0].split(' ')
    new_output.append(words[0])

else:
    new_output = output
            
new_output

['Abstract',
 '1 Introduction',
 '2 Background and Definitions',
 '3 Challenges and open questions',
 '4 Available MT datasets for ILA',
 '5 Low-resource MT',
 '6 Low-resource MT paradigms',
 '7 Advances in MT for the indigenous languages of the Americas',
 '8 Ethical aspects',
 '9 Conclusion']

In [57]:
for i in range(1,len(output)):
  if(output != new_output):
    element = output[i].split('.')[0][:-1]
    element_new = ''
    if(output[i].split('.')[0][-1].isdigit()):
        element_new = element
    else:
        element_new = output[i].split('.')[0]
        
    new_output.append(element_new.rstrip())
  else:
     pass

new_output

         

['Abstract',
 '1 Introduction',
 '2 Background and Definitions',
 '3 Challenges and open questions',
 '4 Available MT datasets for ILA',
 '5 Low-resource MT',
 '6 Low-resource MT paradigms',
 '7 Advances in MT for the indigenous languages of the Americas',
 '8 Ethical aspects',
 '9 Conclusion']

In [58]:
sections = new_output
len(sections)

10

In [59]:
# Extracting text falling under each sections for summary
section_extraction = []

for i in range(len(sections)-1):
    start_index = text_without_urls.find(sections[i])
    end_index = text_without_urls.find(sections[i+1])

    if start_index != -1 and end_index != -1:
        extraction = text_without_urls[start_index:end_index].strip()
        print("Markers found in the text.",sections[i])
        section_extraction.append(extraction)
    else:
        print("Markers not found in the text.",sections[i])

# Extract the last section separately
last_start_index = text_without_urls.find(sections[-1])
if last_start_index != -1:
    last_extraction = text_without_urls[last_start_index:].strip()
    print("Markers found in the text.", sections[-1])
    section_extraction.append(last_extraction)
else:
    print("Markers not found in the text.", sections[-1])

Markers found in the text. Abstract
Markers found in the text. 1 Introduction
Markers found in the text. 2 Background and Definitions
Markers found in the text. 3 Challenges and open questions
Markers found in the text. 4 Available MT datasets for ILA
Markers found in the text. 5 Low-resource MT
Markers not found in the text. 6 Low-resource MT paradigms
Markers not found in the text. 7 Advances in MT for the indigenous languages of the Americas
Markers found in the text. 8 Ethical aspects
Markers found in the text. 9 Conclusion


In [60]:
section_extraction

['Abstract\nNeural models have drastically advanced state\nof the art for machine translation (MT) be-\ntween high-resource languages. Traditionally,\nthese models rely on large amounts of train-\ning data, but many language pairs lack these\nresources. However, an important part of the\nlanguages in the world do not have this amount\nof data. Most languages from the Americas are\namong them, having a limited amount of par-\nallel and monolingual data, if any. Here, we\npresent an introduction to the interested reader\nto the basic challenges, concepts, and tech-\nniques that involve the creation of MT systems\nfor these languages. Finally, we discuss the\nrecent advances and findings and open ques-\ntions, product of an increased interest of the\nNLP community in these languages.',
 '1 Introduction\nMore than 7 billion people on Earth communicate\nin nearly 7000 different languages (Pereltsvaig,\n2020). Of these, approximately 900 languages\nare native of the American continent (Campb

### Applying LSA for summarizing the section extraction

In [61]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [62]:
# Vectorizing the extracted sentences
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(section_extraction)

# 3. Create a TruncatedSVD object for LSA 
lsa = TruncatedSVD(n_components = 7)

In [63]:
# 4. Perform LSA on the TF-IDF matrix
lsa_matrix = lsa.fit_transform(tfidf_matrix)
print(lsa_matrix)

[[ 0.70072576  0.54734714  0.0060138  -0.19535958 -0.35470592  0.03044179
  -0.16467642]
 [ 0.83094813  0.20731302 -0.01527954 -0.11648635  0.060489    0.01919389
   0.20940075]
 [ 0.78959341 -0.21578694 -0.07652416 -0.05654078  0.15016734  0.50406263
  -0.2103922 ]
 [ 0.82983254 -0.11145451 -0.11039983 -0.18104609  0.03488119  0.01990331
   0.4042745 ]
 [ 0.71021895 -0.46393653 -0.31428962 -0.09523508 -0.26755229 -0.26675063
  -0.14988971]
 [ 0.77979426  0.08682592  0.1763532  -0.12868583  0.44622058 -0.30494595
  -0.19454685]
 [ 0.67136877 -0.22502615  0.63553244  0.22458846 -0.20738742  0.00132172
   0.03534088]
 [ 0.6782175   0.17683355 -0.26608363  0.65727339  0.04727918 -0.03751354
   0.00954011]]


In [64]:
import numpy as np

# Calculate LSA scores for each sentence
lsa_scores = np.sum(np.abs(lsa_matrix), axis=1)

# Group sentences by section and rank within each group
grouped_sentences = {}
for sentence, section, score in zip(section_extraction, sections, lsa_scores):
    grouped_sentences.setdefault(section, []).append((score, sentence))
ranked_sentences = {
    section: sorted(sentences, key=lambda x: x[0], reverse=True)
    for section, sentences in grouped_sentences.items()
}

# Choose the top-ranked sentences from each section
num_top_sentences = 5
top_ranked_sentences = []

# Iterate through each section and extract the top-ranked sentences
for section, sentences in ranked_sentences.items():
    top_sentences = sentences[:num_top_sentences]
    top_ranked_sentences.extend(sentence[1] for sentence in top_sentences)
    print(f"Top-ranked sentences for Section {section}:\n" + ''.join(sentence[1] + '\n' for sentence in top_sentences) + '\n')


Top-ranked sentences for Section Abstract:
Abstract
Neural models have drastically advanced state
of the art for machine translation (MT) be-
tween high-resource languages. Traditionally,
these models rely on large amounts of train-
ing data, but many language pairs lack these
resources. However, an important part of the
languages in the world do not have this amount
of data. Most languages from the Americas are
among them, having a limited amount of par-
allel and monolingual data, if any. Here, we
present an introduction to the interested reader
to the basic challenges, concepts, and tech-
niques that involve the creation of MT systems
for these languages. Finally, we discuss the
recent advances and findings and open ques-
tions, product of an increased interest of the
NLP community in these languages.


Top-ranked sentences for Section 1 Introduction:
1 Introduction
More than 7 billion people on Earth communicate
in nearly 7000 different languages (Pereltsvaig,
2020). Of these, appr

### Challenges to solve by next week:
1) Extracting section headers where subheadings are not properly formatted within PDF.
2) Even after choosing num of sentences threshold for summary, it is not working.
3) Implementing LLM APIs for summarization on extracted and processed text.